# Digitale Medien

In [1]:
# loading libs
import pandas as pd
import numpy as np
import pprint as pp
import datetime
import Stemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer

In [2]:
# politiker = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/politikR_demojized_reinigung.csv",
#                         index_col=0)

# öffentlicher = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/örr_demojized.csv",
#                            index_col=0)

# überregionale = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/überregio_demojized.csv",
#                             index_col=0)
# überregionale = überregionale.drop(columns='...1')

digitale = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/digitale_demojized.csv",
                       index_col=0)
digitale = digitale.drop(columns='...1')

In [3]:
digitale

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,hashtagList,cashtags,user,userID,userLoc,followerAmount,erreichbarkeit,normenWerte,geschäftsmodell,text
0,2022-10-27T06:26:03Z,1.585518e+18,@CDU Seit Jahresbeginn haben Parteien insg. 10...,3,13,65,3,NaN,NaN,NaN,...,NaN,NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,@cdu seit jahresbeginn haben parteien insg 10 ...
1,2022-10-26T16:57:38Z,1.585315e+18,Die umstrittene Klimastiftung MV möchte verhin...,4,55,129,5,NaN,NaN,NaN,...,['NordStream2'],NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,die umstrittene klimastiftung mv möchte verhin...
2,2022-10-26T16:20:51Z,1.585305e+18,"""Bitte gehen Sie weiter, es gibt hier nichts z...",1,19,56,0,NaN,NaN,NaN,...,NaN,NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,bitte gehen sie weiter es gibt hier nichts zu ...
3,2022-10-26T14:52:11Z,1.585283e+18,+++ Großspenden-Ticker +++\n\n55.000 Euro an d...,5,50,280,1,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN,NaN,...,NaN,NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,großspenden-ticker \n\n55000 euro an die @cdu...
4,2022-10-26T12:44:57Z,1.585251e+18,Hier haben wir 2021 die Hintergründe beschrieb...,2,25,136,1,NaN,NaN,NaN,...,NaN,NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,hier haben wir 2021 die hintergründe beschrieb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89155,2022-01-01T10:26:52Z,1.477225e+18,"""Dachte, du würdest ewig leben"": So trauern di...",0,3,6,0,NaN,NaN,NaN,...,NaN,NaN,WEBDENews,68724671.0,München,4695,1.0,1.0,2.0,dachte du würdest ewig leben so trauern die st...
89156,2022-01-01T09:51:06Z,1.477216e+18,Deutscher Student bei Messerstecherei in Tel A...,0,1,1,0,NaN,NaN,NaN,...,NaN,NaN,WEBDENews,68724671.0,München,4695,1.0,1.0,2.0,deutscher student bei messerstecherei in tel a...
89157,2022-01-01T08:14:34Z,1.477192e+18,Drosten vorsichtig optimistisch für weitere Co...,0,0,1,0,NaN,NaN,NaN,...,NaN,NaN,WEBDENews,68724671.0,München,4695,1.0,1.0,2.0,drosten vorsichtig optimistisch für weitere co...
89158,2022-01-01T07:28:32Z,1.477180e+18,Polizei zieht nach Silvesternacht Bilanz - Töd...,0,0,1,0,NaN,NaN,NaN,...,NaN,NaN,WEBDENews,68724671.0,München,4695,1.0,1.0,2.0,polizei zieht nach silvesternacht bilanz tödl...


In [4]:
categoriesDigital = {
    'proteste' : ['proteste', 'widerstand', 'corona-maßnahmen', 'protest', 'corona-politik', 'afghanistan'],
    
    'unglück' : ['unfall', 'verletzt', 'einsatz', 'tote', 'feuerwehr', 'verletzte', 'stirbt', 'polizisten',
                 'a45-sperrung', 'feuer', 'brand'],
    
    'covid' : ['corona', 'lauterbach', 'impfpflicht', 'pandemie', 'impfung', 'maskenpflicht',
               'gesundheitsminister', 'omikron', 'maske', 'impfen', 'rki', 'nebenwirkungen', 'ungeimpfte',
               'masken', 'geimpft', 'coronavirus', 'drosten'],
    
    'öpnvVerkehr' : ['bahn', 'fahren', '9-euro-ticket', 'urlaub', 'chaos'],
    
    'wetterKlima' : ['sommer', 'wasser', 'klimawandel', 'wetter', 'hitze', 'grad', 'temperaturen'],
    
    'europa' : ['polen', 'europa', 'österreich', 'europäische', 'spanien', 'ungarn', 'brüssel'],
    
    'kinderbetreuung' : ['kinder', 'junge', 'eltern', 'kindern', 'jungen', 'schulen', 'jugendliche',
                         'schule', 'mädchen', 'schüler'],
    
    'energiekrise' : ['gas', 'habeck', 'bundesregierung', 'energie', 'energiekrise', 'energiewende', 'öl',
                      'wirtschaftsminister', 'atomkraft', 'erdgas', 'strom', 'gasumlage', 'sparen',
                      'gaskrise', 'atomkraftwerke', 'inflation', 'preise', 'steigen', 'steigt', 'teurer',
                      'kosten', 'energiepreise', 'preis', 'kunden', 'steigende', 'lebensmittel'],
    
    'ukraine' : ['ukraine', 'russische', 'russischen', 'ukraine-krieg', 'russland', 'ukrainische', 'kiew',
                 'angriff', 'selenskyj', 'ukrainischen', 'soldaten', 'russischer', 'truppen', 'armee', 'ukrainer',
                 'russlands', 'russen', 'mariupol', 'ukrainekrieg', 'ukraine', 'russland', 'putin', 'krieg',
                 'putins', 'sanktionen', 'nato', 'waffen', 'frieden', 'moskau', 'ukrainekrieg', 'russlands',
                 'kreml', 'waffenlieferungen', 'russen', 'krieges'],
    
    'fakenews' : ['querdenker', 'falsch', 'lügen', 'fehler', 'wahrheit', 'fake', 'hass', 'news',
                  'propaganda', 'meinung', 'fakten', 'gegenteil', 'desinformation'],
    
    'finanzielles' : ['euro', 'millionen', 'geld', 'milliarden', '€', 'wert', 'spenden', 'dollar', 'kosten', 'mio']
}

In [5]:
dfsDigitale = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categoriesDigital.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsDigitale.append(words)

termsDigitale = pd.concat(dfsDigitale)
termsDigitale

,category,term
0,proteste,proteste
1,proteste,widerstand
2,proteste,corona-maßnahmen
3,proteste,protest
4,proteste,corona-politik
...,...,...
5,finanzielles,wert
6,finanzielles,spenden
7,finanzielles,dollar
8,finanzielles,kosten


In [7]:
# Take the 'term' column from our list of terms
termlistDigital = list(set(termsDigitale.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistDigital)

matrix = vectorizer.fit_transform(digitale.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [8]:
# Group the terms by category, then loop through each category
for category_name, rows in termsDigitale.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    digitale[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covid 
Worte: ['corona', 'lauterbach', 'impfpflicht', 'pandemie', 'impfung', 'maskenpflicht', 'gesundheitsminister', 'omikron', 'maske', 'impfen', 'rki', 'nebenwirkungen', 'ungeimpfte', 'masken', 'geimpft', 'coronavirus', 'drosten'] 

Topic: energiekrise 
Worte: ['gas', 'habeck', 'bundesregierung', 'energie', 'energiekrise', 'energiewende', 'öl', 'wirtschaftsminister', 'atomkraft', 'erdgas', 'strom', 'gasumlage', 'sparen', 'gaskrise', 'atomkraftwerke', 'inflation', 'preise', 'steigen', 'steigt', 'teurer', 'kosten', 'energiepreise', 'preis', 'kunden', 'steigende', 'lebensmittel'] 

Topic: europa 
Worte: ['polen', 'europa', 'österreich', 'europäische', 'spanien', 'ungarn', 'brüssel'] 

Topic: fakenews 
Worte: ['querdenker', 'falsch', 'lügen', 'fehler', 'wahrheit', 'fake', 'hass', 'news', 'propaganda', 'meinung', 'fakten', 'gegenteil', 'desinformation'] 

Topic: finanzielles 
Worte: ['euro', 'millionen', 'geld', 'milliarden', '€', 'wert', 'spenden', 'dollar', 'kosten', 'mio'] 

Top

In [9]:
digitale.groupby('user').ukraine.sum().sort_values(ascending=False)

user
WEBDENews          1309
derfreitag          896
tonline             893
TAG24               734
telepolis_news      648
COMPACTMagazin      621
Weltwoche           564
cicero_online       527
Volksverpetzer      459
TichysEinblick      454
krautreporter       449
svz_de              404
NNNonline           373
talkimhangar7       349
jreichelt           334
reitschuster        323
OvertonMagazin      305
pressenza_ger       253
krassundkonkret     243
NachDenkSeiten      240
ruhrbarone          209
Achgut_com          198
correctiv_org       179
LRM_dietalkshow     171
infosperber         162
inFranken           160
HalleSpektrum       139
comeon_de           123
apolut_net           82
Verfassungsblog      66
lobbycontrol         56
theeuropean          48
uebermedien          42
nordbayern           41
stv_klartext         40
a_watch              36
ServusTV_News        17
NuoVisoTV            12
JungNaiv              8
langemannmedien       5
acTVismMunich         1
mucbook    

In [10]:
digitale.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,energiekrise,europa,fakenews,finanzielles,kinderbetreuung,proteste,ukraine,unglück,wetterKlima,öpnvVerkehr
user,,,,,,,,,,,,,,,,,,,,,
Achgut_com,2022-10-31T17:10:05Z2022-10-31T13:50:31Z2022-1...,2.522169e+21,#zweigeschlechter #Biologie #fakten(Pieter Cle...,7410,27653,109272,1637,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AliCologne/status/15867885...,...,148,40,78,47,42,18,198,10,26,7
COMPACTMagazin,2022-10-31T20:52:31Z2022-10-31T17:53:17Z2022-1...,3.932345e+21,Wenn es um den besonderen #Schutz fürs #herz g...,5629,18300,57490,1500,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/maxotte_says/status/149645...,...,337,87,96,114,20,63,621,22,27,11
GMX,2022-10-24T14:28:11Z2022-10-21T08:46:06Z2022-1...,6.250115e+19,Viele große Dateien per E-Mail im #Anhang vers...,23,39,77,9,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/DeutschePostDHL/status/156...,...,1,0,3,1,0,0,0,1,0,0
HalleSpektrum,2022-10-31T19:42:16Z2022-10-31T15:40:15Z2022-1...,3.682287e+21,Im Walde steht ein Pilz Wer hat sie auf seinen...,79,177,538,55,0,0.0,0,...,148,6,19,77,92,23,139,70,45,15
Hintergrund_de,2022-10-31T07:37:30Z2022-10-31T07:28:23Z2022-1...,7.934324e+18,Kinder und Jugendliche sollen frühzeitig mit d...,0,2,1,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,0,0,1,0,1,0,1,0,0,0
JungNaiv,2022-10-26T17:41:51Z2022-10-26T14:15:49Z2022-1...,1.472613e+20,Here we go: https://t.co/g1CJI22RUORegierungst...,443,618,3878,95,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/TiloJung/status/1584588474...,...,7,1,1,0,1,1,8,0,0,1
LRM_dietalkshow,2022-10-24T09:19:46Z2022-10-24T06:16:00Z2022-1...,6.556213e+20,Best of #LRM mit @mfleischhacker1\nWie #korrup...,1491,2764,11110,440,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/LRM_dietalkshow/status/157...,...,52,28,22,11,3,0,171,1,1,0
NNNonline,2022-10-31T19:00:08Z2022-10-30T14:30:12Z2022-1...,5.387091e+21,Die Rostocker #Eisbademeisters eröffnen die Ei...,307,263,1916,65,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,296,40,31,195,152,21,373,113,63,70
NachDenkSeiten,2022-10-31T13:01:08Z2022-10-31T11:04:19Z2022-1...,2.134005e+21,"Politik ohne Gott – Oder: Erst der Bürger, dan...",3930,13498,39975,1198,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/FWarweg/status/15668233164...,...,35,20,30,13,6,9,240,11,8,6


In [12]:
digitaleCategorized = digitale
digitaleCategorized.to_csv("../../methods/representative-speech/mediendaten/zuordnungenBundesländer/digitaleCategorized.csv")